## Coursera Capstone Project 
### Customer Segmentation and Clustering based on Location Amenities

In [1]:
import pandas as pd
import numpy as np

print("Hello Capstone Project Course")

Hello Capstone Project Course


In [24]:
pip install lxml

Note: you may need to restart the kernel to use updated packages.


In [13]:
pip install bs4

  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1279 sha256=e99446f77502fa30499e481cd5a3e2f088a1c7fa64b3b41e1ce2ffcf8fda277a
  Stored in directory: c:\users\visha\appdata\local\pip\cache\wheels\75\78\21\68b124549c9bdc94f822c02fb9aa3578a669843f9767776bca
Successfully built bs4
Note: you may need to restart the kernel to use updated packages.


In [10]:
pip install tabulate

  Created wheel for tabulate: filename=tabulate-0.8.6-py3-none-any.whl size=23274 sha256=21b15d871f286de389368a19bc5d752f800fef9ff52ecab620e018a3cd9c56ee
  Stored in directory: c:\users\visha\appdata\local\pip\cache\wheels\86\8c\d9\3b6c2f55efaefe656c52b725d2fdc5669085c83444979db8b3
Successfully built tabulate
Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install beautifulsoup4

  Using cached beautifulsoup4-4.8.2-py3-none-any.whl (106 kB)
  Using cached soupsieve-2.0-py2.py3-none-any.whl (32 kB)



In [3]:
pip install requests

  Using cached requests-2.23.0-py2.py3-none-any.whl (58 kB)
  Using cached urllib3-1.25.8-py2.py3-none-any.whl (125 kB)
  Using cached chardet-3.0.4-py2.py3-none-any.whl (133 kB)
  Using cached idna-2.9-py2.py3-none-any.whl (58 kB)



In [65]:
pip install csv

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement csv (from versions: none)
ERROR: No matching distribution found for csv


## 1. Read url and scrape required data from using BeautifulSoup library

In [142]:
# Import libraries

import pandas as pd
import numpy as np
import requests
import csv
from bs4 import BeautifulSoup

# Read source URL & Parse using lxml
source = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
soup = BeautifulSoup(source, 'lxml')

CSV_FILE_NAME = 'cms_scrape.csv'

# Open CSV File
csv_file = open(CSV_FILE_NAME,'w',newline='')

csv_writer = csv.writer(csv_file)
# print(type(csv_writer))

# Write header column names to the file
csv_writer.writerow(['Postal Code', 'Borough', 'Neighbourhood'])

# Loop through table rows
for tr in soup.find_all('tr'):
    
## For each row traverse to each columns    
    for td in tr.find_all('td'):
        
## Read Postal Code        
        try:
            post_code = td.p.b.text                    
        except Exception as e:
            post_code = None
#         print('Postal Code :', post_code)    

## Read Borough
        try:
            borough = td.p.span.a.text
            if borough:
                borough = td.p.span.text
        except Exception as e:
            borough = None
#         print('Borough :', borough)
        
## Read Neighbourhoood    
        try:
            first_link = td.p.span.a            
            neighbour = []
            for a in first_link.find_next_siblings("a"):
                if a.text:
                  neighbour.append(a.text)          
        except Exception as e:
            neighbour = None
            
## Seprate neighbourhood with comma for multiple values        
        try:            
            neighbour_str = ', '.join([str(x) for x in neighbour])
        except Exception as e:
            neighbour_str = None
        
#         print('Neighbour :',neighbour_str)        

## Write the row record to file
        csv_writer.writerow([str(post_code), str(borough), str(neighbour_str)])

csv_file.close()    


## 2. Read the data from CSV File to Dataframe

In [143]:
df = pd.read_csv(CSV_FILE_NAME)
df.shape

(245, 3)

In [144]:
df = df[df['Borough']!= 'None']
df.shape

(101, 3)

In [145]:
df.head(101)

,Postal Code,Borough,Neighbourhood
2,M3A,North York(Parkwoods),Parkwoods
3,M4A,North York(Victoria Village),Victoria Village
4,M5A,Downtown Toronto(Regent Park / Harbourfront),"Regent Park, Harbourfront"
5,M6A,North York(Lawrence Manor / Lawrence Heights),"Lawrence Manor, Lawrence Heights"
6,M7A,Queen's Park / Ontario Provincial Government,Ontario Provincial Government
...,...,...,...
160,M8X,Etobicoke(The Kingsway / Montgomery Road / Old...,"The Kingsway, Old Mill"
165,M4Y,Downtown Toronto(Church and Wellesley),Church and Wellesley
168,M7Y,East TorontoBusiness reply mail Processing Cen...,NaN
169,M8Y,Etobicoke(Old Mill South / King's Mill Park / ...,"Old Mill, Sunnylea, Humber Bay, Mimico, The Qu..."


In [146]:
# Setting the Neighbourhood values with Borough where Neighbourhood is Not Assigned

df.isna().sum()

Postal Code       0
Borough           0
Neighbourhood    13
dtype: int64

In [147]:
df['Neighbourhood'].fillna(df['Borough'], inplace=True)
df.head(50)

,Postal Code,Borough,Neighbourhood
2,M3A,North York(Parkwoods),Parkwoods
3,M4A,North York(Victoria Village),Victoria Village
4,M5A,Downtown Toronto(Regent Park / Harbourfront),"Regent Park, Harbourfront"
5,M6A,North York(Lawrence Manor / Lawrence Heights),"Lawrence Manor, Lawrence Heights"
6,M7A,Queen's Park / Ontario Provincial Government,Ontario Provincial Government
8,M9A,Etobicoke(Islington Avenue),Islington Avenue
9,M1B,Scarborough(Malvern / Rouge),"Malvern, Rouge"
11,M3B,North York(Don Mills)North,Don Mills
12,M4B,East York(Parkview Hill / Woodbine Gardens),"Parkview Hill, Woodbine Gardens"
13,M5B,"Downtown Toronto(Garden District, Ryerson)","Garden District, Ryerson"


In [148]:
# df = df[df['Postal Code']=='M4M']
# df

In [149]:
df_new = df[df['Postal Code'].isin(['M5G', 'M2H','M4B','M1J','M4G','M4M','M1R','M9V','M9L','M5V','M1B','M5A'])]
df_new.reset_index(inplace=True, drop=True)
df_new

,Postal Code,Borough,Neighbourhood
0,M5A,Downtown Toronto(Regent Park / Harbourfront),"Regent Park, Harbourfront"
1,M1B,Scarborough(Malvern / Rouge),"Malvern, Rouge"
2,M4B,East York(Parkview Hill / Woodbine Gardens),"Parkview Hill, Woodbine Gardens"
3,M4G,East York(Leaside),Leaside
4,M5G,Downtown Toronto(Central Bay Street),Bay Street
5,M2H,North York(Hillcrest Village),Hillcrest Village
6,M1J,Scarborough(Scarborough Village),Scarborough Village
7,M9L,North York(Humber Summit),Humber Summit
8,M1R,Scarborough(Wexford / Maryvale),"Wexford, Maryvale"
9,M5V,Downtown Toronto(CN Tower / King and Spadina /...,"CN Tower, King and Spadina, Railway Lands, Har..."


## Geo Code Data for Postal Codes

In [150]:
# import geocoder # import geocoder

# # initialize your variable to None
# lat_lng_coords = None

# # loop until you get the coordinates
# while(lat_lng_coords is None):
#   g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
#   lat_lng_coords = g.latlng

# latitude = lat_lng_coords[0]
# longitude = lat_lng_coords[1]

df_geo_codes = pd.read_csv('Geospatial_Coordinates.csv')
df_geo_codes.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [151]:
df_geo = df.join(df_geo_codes.set_index('Postal Code'),on='Postal Code')
df_geo

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
2,M3A,North York(Parkwoods),Parkwoods,43.753259,-79.329656
3,M4A,North York(Victoria Village),Victoria Village,43.725882,-79.315572
4,M5A,Downtown Toronto(Regent Park / Harbourfront),"Regent Park, Harbourfront",43.654260,-79.360636
5,M6A,North York(Lawrence Manor / Lawrence Heights),"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
6,M7A,Queen's Park / Ontario Provincial Government,Ontario Provincial Government,43.662301,-79.389494
...,...,...,...,...,...
160,M8X,Etobicoke(The Kingsway / Montgomery Road / Old...,"The Kingsway, Old Mill",43.653654,-79.506944
165,M4Y,Downtown Toronto(Church and Wellesley),Church and Wellesley,43.665860,-79.383160
168,M7Y,East TorontoBusiness reply mail Processing Cen...,East TorontoBusiness reply mail Processing Cen...,43.662744,-79.321558
169,M8Y,Etobicoke(Old Mill South / King's Mill Park / ...,"Old Mill, Sunnylea, Humber Bay, Mimico, The Qu...",43.636258,-79.498509


In [152]:
df_geo_new = df_new.join(df_geo_codes.set_index('Postal Code'),on='Postal Code')
df_geo_new

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto(Regent Park / Harbourfront),"Regent Park, Harbourfront",43.654260,-79.360636
1,M1B,Scarborough(Malvern / Rouge),"Malvern, Rouge",43.806686,-79.194353
2,M4B,East York(Parkview Hill / Woodbine Gardens),"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
3,M4G,East York(Leaside),Leaside,43.709060,-79.363452
4,M5G,Downtown Toronto(Central Bay Street),Bay Street,43.657952,-79.387383
5,M2H,North York(Hillcrest Village),Hillcrest Village,43.803762,-79.363452
6,M1J,Scarborough(Scarborough Village),Scarborough Village,43.744734,-79.239476
7,M9L,North York(Humber Summit),Humber Summit,43.756303,-79.565963
8,M1R,Scarborough(Wexford / Maryvale),"Wexford, Maryvale",43.750072,-79.295849
9,M5V,Downtown Toronto(CN Tower / King and Spadina /...,"CN Tower, King and Spadina, Railway Lands, Har...",43.628947,-79.394420
